In [1]:
import pandas as pd
import dask.dataframe as dd

In [2]:
msa_src = pd.read_csv('nzlud_msa_orig.csv', encoding='latin')
muni_src = pd.read_csv('nzlud_muni_orig.csv', encoding='latin')

# msa_src = msa_src.rename(columns={'zri_full_st': 'orig_zri_full_st'})
# muni_src = muni_src.rename(columns={'zri_st': 'orig_zri_st'})


In [3]:
msa_comp = pd.read_csv('other_data/processed_dana/nzlud_msa.csv')
muni_comp = pd.read_csv('other_data/processed_dana/nzlud_muni.csv')

# msa_src = msa_src.rename(columns={'zri_full_st': 'new_zri_full_st'})
# muni_src = muni_src.rename(columns={'zri_st': 'new_zri_st'})

In [4]:
len(muni_src)

2639

In [5]:
len(muni_comp)

2638

In [11]:
# Compute an average ZRI at the state level
muni_comp = muni_comp.groupby('statename').agg({'zri_up_st': 'mean'})
muni_comp = muni_comp.sort_values('zri_up_st', ascending=False)
display(muni_comp)

,zri_up_st
statename,
WA,1.218020
NV,1.145765
CA,0.964029
RI,0.929752
NH,0.688416
OR,0.577292
MD,0.565382
WY,0.559193
NJ,0.494899


In [6]:
muni_comp['GEOID'].is_unique

True

In [7]:
muni_comp.columns

Index(['GEOID', 'statename', 'place', 'type', 'restrict_sf_permit',
       'restrict_mf_permit', 'limit_sf_units', 'limit_mf_units',
       'limit_mf_dwellings', 'limit_mf_dwelling_units', 'min_lot_size',
       'open_space', 'inclusionary', 'half_acre_less', 'half_acre_more',
       'one_acre_more', 'two_acre_more', 'max_den_cat1', 'max_den_cat2',
       'max_den_cat3', 'max_den_cat4', 'max_den_cat5', 'council_nz',
       'planning_nz', 'countybrd_nz', 'pubhlth_nz', 'site_plan_nz',
       'env_rev_nz', 'council_rz', 'planning_rz', 'zoning_rz', 'countybrd_rz',
       'countyzone_rz', 'townmeet_rz', 'env_rev_rz', 'total_nz', 'total_rz',
       'adu', 'height_ft_median', 'height_ft_mode', 'height_st_median',
       'height_st_mode', 'parking_median', 'parking_mode', 'mf_per', 'maxden5',
       'maxden4', 'maxden3', 'maxden2', 'maxden1', 'sindex1', 'sindex2',
       'sindex3', 'sindex4', 'sindex5', 'sindex6', 'sindex7', 'sindex8',
       'height_st_median_sc', 'height_st_mode_sc', 'sindex

In [8]:
msa_src.equals(msa_comp)

False

In [9]:
muni_src.equals(muni_comp)

False

In [10]:
msa_df = pd.merge(
    msa_src, 
    msa_comp, 
    how='inner', 
    on=['cbsa10'], 
    suffixes=('_orig', '_new')
)
muni_df = pd.merge(
    muni_src, 
    muni_comp, 
    how='inner', 
    on=['GEOID', 'timestamp'], 
    suffixes=('_orig', '_new')
)


In [11]:
for ind in (msa_src.loc[msa_src['zri_full'] != msa_comp['zri_full']].index):
    x =  msa_src.loc[msa_src.index == ind, 'zri_full'].values[0]
    y =  msa_comp.loc[msa_comp.index == ind, 'zri_full'].values[0]
    print(x, y)

ValueError: Can only compare identically-labeled Series objects

In [ ]:
msa_df = msa_df[['cbsa10', 'cbsaname10_orig', 'zri_full_orig', 'zri_full_new']]

In [ ]:
muni_df = muni_df[['statename_orig', 'place_orig', 'zri_st_orig', 'zri_st_new']]

In [ ]:
display(msa_df)

,cbsa10,cbsaname10_orig,zri_full_orig,zri_full_new
0,10180,"Abilene, TX Metropolitan Statistical Area",NaN,NaN
1,10420,"Akron, OH Metropolitan Statistical Area",3.194724,3.192274
2,10500,"Albany, GA Metropolitan Statistical Area",NaN,NaN
3,10540,"Albany-Lebanon, OR Micropolitan Statistical Area",0.341198,0.339414
4,10580,"Albany-Schenectady-Troy, NY Metropolitan Stati...",NaN,NaN
...,...,...,...,...
317,49340,"Worcester, MA Metropolitan Statistical Area",NaN,NaN
318,49620,"York-Hanover, PA Metropolitan Statistical Area",5.253992,5.253197
319,49660,"Youngstown-Warren-Boardman, OH-PA Metropolitan...",NaN,NaN
320,49700,"Yuba City, CA Metropolitan Statistical Area",1.861072,1.859132


In [ ]:
display(muni_df)

,statename_orig,place_orig,zri_st_orig,zri_st_new
0,AL,Athens,-1.231126,-1.230618
1,AL,Auburn,0.199853,0.200040
2,AL,Bessemer,0.268941,0.268692
3,AL,Birmingham,-0.097643,-0.098133
4,AL,Chickasaw,-0.151553,-0.151482
...,...,...,...,...
2633,WY,Douglas,1.603348,1.603167
2634,WY,Jackson,2.023841,2.023341
2635,WY,Lander,-0.518137,-0.518306
2636,WY,Laramie,1.021414,1.021630


In [ ]:
msa_df.to_csv('msa_comp.csv')

In [ ]:
muni_df.to_csv('muni_comp.csv')